<a href="https://colab.research.google.com/github/BenBerol/CSProjects/blob/main/ClimbingBiomechanics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe==0.10.9 opencv-python-headless pandas matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
# CSV Creator
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

# Upload video
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# Set up MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Read video
cap = cv2.VideoCapture(video_path)
pose_data = []

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks:
        row = []
        for lm in results.pose_landmarks.landmark:
            row.extend([lm.x, lm.y])
        pose_data.append(row)

cap.release()

# Get column names
landmark_names = [l.name.lower() for l in mp_pose.PoseLandmark]
columns = []
for name in landmark_names:
    columns.extend([f"{name}_x", f"{name}_y"])

df = pd.DataFrame(pose_data, columns=columns)
df.to_csv("pose_data.csv", index=False)
files.download("pose_data.csv")

print("✅ Pose data extracted and saved as CSV.")


Saving benthrow.mov to benthrow.mov


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Pose data extracted and saved as CSV.


In [ ]:
#Skeleton Overlay Creator

from google.colab import files
uploaded = files.upload()  # Upload your .mp4 video AND pose_data.csv

import cv2
import pandas as pd
import numpy as np

video_path = [f for f in uploaded if f.lower().endswith(('.mp4', '.mov'))][0]
csv_path = [f for f in uploaded if f.lower().endswith('.csv')][0]

cap = cv2.VideoCapture(video_path)
pose_df = pd.read_csv(csv_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define landmark names and drawing connections
landmarks = [col[:-2] for col in pose_df.columns[::2]]
connections = [
    ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
    ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
    ("left_shoulder", "right_shoulder"),
    ("left_hip", "right_hip"),
    ("left_shoulder", "left_hip"), ("right_shoulder", "right_hip"),
    ("left_hip", "left_knee"), ("left_knee", "left_ankle"),
    ("right_hip", "right_knee"), ("right_knee", "right_ankle")
]

out = cv2.VideoWriter("annotated_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
frame_idx = 0

while cap.isOpened() and frame_idx < len(pose_df):
    ret, frame = cap.read()
    if not ret:
        break

    row = pose_df.iloc[frame_idx]

    # Convert normalized coordinates to pixel coordinates

    excluded_joints = {
      'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer',
      'right_eye_inner', 'right_eye', 'right_eye_outer',
      'left_ear', 'right_ear',
      'mouth_left', 'mouth_right',
      'left_index', 'right_index',
      'left_thumb', 'right_thumb',
      'left_pinky', 'right_pinky',
      'left_foot_index', 'right_foot_index',
      'left_heel', 'right_heel'
    }

    coords = {}
    for name in landmarks:
        if name in excluded_joints:
            continue
        x = int(row[f"{name}_x"] * width)
        y = int(row[f"{name}_y"] * height)
        coords[name] = (x, y)
        cv2.circle(frame, (x, y), 4, (0, 0, 255), -1)  # red dot

    # Draw connections
    for a, b in connections:
        if a in coords and b in coords:
            cv2.line(frame, coords[a], coords[b], (0, 255, 0), 2)  # green line

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()

files.download("annotated_video.mp4")
print("✅ Annotated video is ready for download.")


Saving bendifclimbdata.csv to bendifclimbdata.csv
Saving bendifficultclimb.mov to bendifficultclimb (1).mov


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Annotated video is ready for download.


In [ ]:
# Combined Pose Skeleton + Angle Overlay Creator

import cv2
import pandas as pd
import numpy as np
from google.colab import files

# === Upload Video + CSV ===
uploaded = files.upload()
video_path = [f for f in uploaded if f.lower().endswith(('.mp4', '.mov'))][0]
csv_path = [f for f in uploaded if f.lower().endswith('.csv')][0]

# === Load Data ===
cap = cv2.VideoCapture(video_path)
df = pd.read_csv(csv_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# === Define Joint Connections ===
connections = [
    ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
    ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
    ("left_hip", "left_knee"), ("left_knee", "left_ankle"),
    ("right_hip", "right_knee"), ("right_knee", "right_ankle"),
    ("left_shoulder", "right_shoulder"), ("left_hip", "right_hip"),
    ("left_shoulder", "left_hip"), ("right_shoulder", "right_hip")
]

# === Joints to Skip (face, hands, feet details) ===
excluded_joints = {
    'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer',
    'right_eye_inner', 'right_eye', 'right_eye_outer',
    'left_ear', 'right_ear', 'mouth_left', 'mouth_right',
    'left_index', 'right_index', 'left_thumb', 'right_thumb',
    'left_pinky', 'right_pinky', 'left_foot_index', 'right_foot_index',
    'left_heel', 'right_heel'
}

# === Angle Calculation ===
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# === Video Writer ===
out = cv2.VideoWriter('skeleton_and_angles.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
frame_idx = 0

while cap.isOpened() and frame_idx < len(df):
    ret, frame = cap.read()
    if not ret:
        break

    row = df.iloc[frame_idx]
    coords = {}
    for col in df.columns[::2]:
        joint = col[:-2]
        if joint in excluded_joints:
            continue
        x = int(row[f"{joint}_x"] * width)
        y = int(row[f"{joint}_y"] * height)
        coords[joint] = (x, y)
        cv2.circle(frame, (x, y), 4, (0, 0, 255), -1)  # red dot

    # Draw connections
    for a, b in connections:
        if a in coords and b in coords:
            cv2.line(frame, coords[a], coords[b], (0, 255, 0), 2)

    # Annotate joint angles
    joints = [
        ("left_shoulder", "left_elbow", "left_wrist", "Elbow (L)"),
        ("right_shoulder", "right_elbow", "right_wrist", "Elbow (R)"),
        ("left_hip", "left_knee", "left_ankle", "Knee (L)"),
        ("right_hip", "right_knee", "right_ankle", "Knee (R)")
    ]

    for a, b, c, label in joints:
        if all(k in coords for k in [a, b, c]):
            angle = int(calculate_angle(coords[a], coords[b], coords[c]))
            cv2.putText(frame, f"{label}: {angle} deg", coords[b], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()

# === Download Result ===
files.download('skeleton_and_angles.mp4')
print("✅ Combined annotated video is ready!")


Saving benthrowdata.csv to benthrowdata (1).csv
Saving benthrow.mov to benthrow (1).mov


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Combined annotated video is ready!


In [ ]:
#Joint Overlay

import cv2
import numpy as np
import pandas as pd
from google.colab import files

# === Upload ===
uploaded = files.upload()  # Upload your .mp4 video and pose_data.csv
video_path = [f for f in uploaded if f.lower().endswith(('.mp4', '.mov'))][0]
csv_path = [f for f in uploaded if f.lower().endswith('.csv')][0]

# === Load Data ===
cap = cv2.VideoCapture(video_path)
df = pd.read_csv(csv_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# === Define Pose Connections ===
connections = [
    ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
    ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
    ("left_hip", "left_knee"), ("left_knee", "left_ankle"),
    ("right_hip", "right_knee"), ("right_knee", "right_ankle"),
    ("left_shoulder", "right_shoulder"), ("left_hip", "right_hip")
]

# === Angle Calculation ===
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# === Video Writer ===
out = cv2.VideoWriter('angle_overlay_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
frame_idx = 0

while cap.isOpened() and frame_idx < len(df):
    ret, frame = cap.read()
    if not ret:
        break

    row = df.iloc[frame_idx]
    coords = {}
    for col in df.columns[::2]:
        joint = col[:-2]
        x = int(row[f"{joint}_x"] * width)
        y = int(row[f"{joint}_y"] * height)
        coords[joint] = (x, y)

    # === Annotate Joint Angles ===
    # Elbow angle (left): shoulder–elbow–wrist
    if all(k in coords for k in ["left_shoulder", "left_elbow", "left_wrist"]):
        angle = calculate_angle(coords["left_shoulder"], coords["left_elbow"], coords["left_wrist"])
        cv2.putText(frame, f"Elbow (L): {int(angle)} d", coords["left_elbow"], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    # Knee angle (left): hip–knee–ankle
    if all(k in coords for k in ["left_hip", "left_knee", "left_ankle"]):
        angle = calculate_angle(coords["left_hip"], coords["left_knee"], coords["left_ankle"])
        cv2.putText(frame, f"Knee (L): {int(angle)} d", coords["left_knee"], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    if all(k in coords for k in ["right_shoulder", "right_elbow", "right_wrist"]):
        angle = calculate_angle(coords["right_shoulder"], coords["right_elbow"], coords["right_wrist"])
        cv2.putText(frame, f"Elbow (R): {int(angle)} d", coords["right_elbow"], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    # Knee angle (left): hip–knee–ankle
    if all(k in coords for k in ["right_hip", "right_knee", "right_ankle"]):
        angle = calculate_angle(coords["right_hip"], coords["right_knee"], coords["right_ankle"])
        cv2.putText(frame, f"Knee (R): {int(angle)} d", coords["right_knee"], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()

# === Download the Result ===
files.download('angle_overlay_video.mp4')
print("✅ Annotated video with joint angles is ready.")


Saving annotated_video (5).mp4 to annotated_video (5) (1).mp4
Saving bendifclimbdata.csv to bendifclimbdata (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Annotated video with joint angles is ready.


In [ ]:
#Climbing Efficiency Calculator

import numpy as np
import pandas as pd
from google.colab import files

uploaded = files.upload()  # Upload your .mp4 video and pose_data.csv
csv_path = [f for f in uploaded if f.lower().endswith('.csv')][0]

df = pd.read_csv(csv_path)

# === Define Pose Connections ===
connections = [
    ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
    ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
    ("left_hip", "left_knee"), ("left_knee", "left_ankle"),
    ("right_hip", "right_knee"), ("right_knee", "right_ankle"),
    ("left_shoulder", "right_shoulder"), ("left_hip", "right_hip")
]

# === Angle Calculation ===
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

frame_idx = 0
climbing_efficiency = []
climb_angles = []
upper_body_efficiency_weights = 1.0
lower_body_efficiency_weights = 1.0

while frame_idx < len(df):

    row = df.iloc[frame_idx]
    coords = {}
    for col in df.columns[::2]:
        joint = col[:-2]
        x = row[f"{joint}_x"]
        y = row[f"{joint}_y"]
        coords[joint] = (x, y)

    left_elbow_angle = calculate_angle(coords["left_shoulder"], coords["left_elbow"], coords["left_wrist"])
    right_elbow_angle = calculate_angle(coords["right_shoulder"], coords["right_elbow"], coords["right_wrist"])
    left_knee_angle = calculate_angle(coords["left_hip"], coords["left_knee"], coords["left_ankle"])
    right_knee_angle = calculate_angle(coords["right_hip"], coords["right_knee"], coords["right_ankle"])

    climb_angles.append(left_elbow_angle)
    climb_angles.append(right_elbow_angle)
    climb_angles.append(left_knee_angle)
    climb_angles.append(right_knee_angle)

    upper_body_efficiency = (min(abs(left_elbow_angle - 180), 90) + min(abs(right_elbow_angle - 180), 90))/2
    lower_body_efficiency = (min(abs(left_knee_angle - 180), 90) + min(abs(right_knee_angle - 180), 90))/2
    inefficiency_score = (upper_body_efficiency*upper_body_efficiency_weights + lower_body_efficiency*lower_body_efficiency_weights);
    normalized_efficiency_score = 1 - (inefficiency_score / (2*90*(upper_body_efficiency_weights+lower_body_efficiency_weights)))
    efficiency_score = normalized_efficiency_score * 100
    climbing_efficiency.append(efficiency_score)


    frame_idx += 1

df["climbing_efficiency"] = climbing_efficiency
df.to_csv("pose_data_with_climbing_efficiency.csv", index=False)
files.download("pose_data_with_climbing_efficiency.csv")
print("✅ Climbing efficiency calculated and saved as CSV.")

avg_angle = np.mean(climb_angles)
print(f"Average Climbing Angle: {avg_angle:.2f} degrees")
avg_efficiency = np.mean(climbing_efficiency)
print(f"Average Climbing Efficiency: {avg_efficiency:.2f}%")

Saving beneasyclimbdata.csv to beneasyclimbdata (5).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Climbing efficiency calculated and saved as CSV.
Average Climbing Angle: 116.96 degrees
Average Climbing Efficiency: 74.54%
